In [1]:
import json
def get_question_id_mapping():
    question_ids = {}

    with open("scraped-data/index.json", "r") as f:
        data = json.load(f)

    counter = 0
    for section in data:
        for subsection in section["subsections"]:
            for article in subsection["articles"]:
                qa = article.get("QA")
                if not qa:
                    break
                for item in qa:
                    counter += 1
                    question = item["question"]
                    question_ids[question] = counter
    return question_ids

question_ids = get_question_id_mapping()

In [16]:
import numpy as np

EMBEDDING_MODEL = "text-embedding-ada-002"
import json
import openai
import numpy as np

def set_pactum_api_key():
    openai.api_key = ''

def set_my_api_key():
    openai.api_key = '' 


import numpy as np

with open('testing/question_embeddings.npy', 'rb') as f:
    question_embeddings = np.load(f)

def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    q_id = question_ids.get(text)
    if q_id:
        return question_embeddings[q_id]
    result = openai.Embedding.create(
        model=model,
        input=text
    )
    return result["data"][0]["embedding"]

def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.

    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))


def code_under_test(question):
    question_embedding = get_embedding(question)

    article_similarities = []

    with open('embeddings/article_embeddings.npy', 'rb') as f:
        article_embeddings = np.load(f)
        
    for article_nr, article_embedding in enumerate(article_embeddings):
        similarity = vector_similarity(question_embedding, article_embedding)
        article_similarities.append((similarity, article_nr))

    article_similarities = sorted(article_similarities, reverse=True)
    return [article_similarities[0][1], article_similarities[1][1]]


In [18]:
# filter for id in the list of objects containing "gpt"

set_my_api_key()
llist = openai.Engine.list()

gpt_engines = [engine for engine in llist.data if "gpt" in engine["id"]]
gpt_engines

[<Engine engine id=gpt-3.5-turbo at 0x1273d4590> JSON: {
   "created": null,
   "id": "gpt-3.5-turbo",
   "object": "engine",
   "owner": "openai",
   "permissions": null,
   "ready": true
 },
 <Engine engine id=gpt-4 at 0x1273d4c20> JSON: {
   "created": null,
   "id": "gpt-4",
   "object": "engine",
   "owner": "openai",
   "permissions": null,
   "ready": true
 },
 <Engine engine id=gpt-4-0314 at 0x1273d4ea0> JSON: {
   "created": null,
   "id": "gpt-4-0314",
   "object": "engine",
   "owner": "openai",
   "permissions": null,
   "ready": true
 },
 <Engine engine id=gpt-3.5-turbo-0301 at 0x127406f70> JSON: {
   "created": null,
   "id": "gpt-3.5-turbo-0301",
   "object": "engine",
   "owner": "openai",
   "permissions": null,
   "ready": true
 }]

In [15]:
gpt_engines

[<Engine engine id=gpt-3.5-turbo at 0x122578a40> JSON: {
   "created": null,
   "id": "gpt-3.5-turbo",
   "object": "engine",
   "owner": "openai",
   "permissions": null,
   "ready": true
 },
 <Engine engine id=gpt-3.5-turbo-0301 at 0x1273d5ad0> JSON: {
   "created": null,
   "id": "gpt-3.5-turbo-0301",
   "object": "engine",
   "owner": "openai",
   "permissions": null,
   "ready": true
 }]

In [22]:
len(question_ids)

1450

In [25]:
def get_next_question():
    with open("scraped-data/index.json", "r") as f:
        data = json.load(f)
    for section in data:
        for subsection in section["subsections"]:
            for article in subsection["articles"]:
                qa = article.get("QA")
                if not qa:
                    continue
                for item in qa:
                    yield item["question"]

import numpy as np

with open('testing/question_embeddings.npy', 'rb') as f:
    question_embeddings = np.load(f)

question_embeddings = list(question_embeddings)

current_length = len(question_embeddings)

for i, question in enumerate(get_next_question()):
    if i < current_length:
        continue
    print(i, question)
    question_embeddings.append(get_embedding(question))
    with open('testing/question_embeddings.npy', 'wb') as f:
        np.save(f, np.array(question_embeddings))



1481 How does Wise verify my business address?
1482 What type of document can I provide to confirm my business address?
1483 Can I submit a utility bill that is older than 3 months to verify my business address?
1484 Can I provide a local authority or FCA-authorised institution document that is older than 3 months?
1485 Can I use a PO Box as my business address?
1486 How can I change the country of my business?
1487 How can I change my personal address on my Wise business account?
1488 How can I edit details on a business account if my business isn't verified?
1489 How do I update my business details on the Android and iOS apps?
1490 Can I update director or shareholder information myself?
1491 How do I edit details on my business account if it is already verified?
1492 Can I switch between different business accounts on Wise?
1493 How do I switch between accounts on the Wise website?
1494 How do I switch between accounts on the Wise mobile app?
1495 Why can't I see the account I'm loo

In [31]:
with open("scraped-data/index.json", "r") as f:
    data = json.load(f)

total = 0
correct = 0
wrong = 0

expected_id = 0
for section in data:
    for subsection in section["subsections"]:
        for article in subsection["articles"]:
            qa = article.get("QA")
            if not qa:
                break
            for item in qa:
                total += 1
                question = item["question"]
                ids = code_under_test(item["question"])
                if expected_id in ids:
                    correct += 1
                else:
                    wrong += 1
                    print(f"Expected {expected_id} but got {ids} for question {question}")
                    print(f"Correct url: {article_id_urls[expected_id]}")
                    print(f"Wrong urls: {article_id_urls[ids[0]]} and {article_id_urls[ids[1]]}")
            expected_id += 1
            

Expected 0 but got [76, 73] for question Can I send money using an email address if I don’t have the recipient's bank details?
Correct url: https://wise.com/help/articles/2977959/how-do-i-send-money-with-wise
Wrong urls: https://wise.com/help/articles/2978030/i-entered-my-recipients-email-incorrectly and https://wise.com/help/articles/2932105/can-i-send-money-to-someone-with-only-their-email-address
Expected 0 but got [247, 2] for question How can I make sure my recipient gets an exact amount?
Correct url: https://wise.com/help/articles/2977959/how-do-i-send-money-with-wise
Wrong urls: https://wise.com/help/articles/2932870/tips-for-paying-invoices and https://wise.com/help/articles/2448314/can-i-send-exact-amounts
Expected 0 but got [11, 10] for question How will I know when my transfer is complete and the money is on its way?
Correct url: https://wise.com/help/articles/2977959/how-do-i-send-money-with-wise
Wrong urls: https://wise.com/help/articles/2977950/why-does-it-say-my-transfer

KeyboardInterrupt: 

In [32]:
# calculate precision and recall using total correct and wrong

accuracy = correct / total

accuracy


0.6613756613756614

In [17]:
article_id_urls

{}

In [6]:
for url, id in article_url_ids.items():
    print(url)

https://wise.com/help/articles/2977959/how-do-i-send-money-with-wise
1
https://wise.com/help/articles/2524078/how-long-does-a-transfer-take
2
https://wise.com/help/articles/2448314/can-i-send-exact-amounts
3
https://wise.com/help/articles/2553293/how-do-you-notify-me-about-a-transfer
4
https://wise.com/help/articles/2977946/how-do-i-download-a-transfer-receipt
5
https://wise.com/help/articles/2565007/do-i-need-to-pay-any-tax-on-transfers
6
https://wise.com/help/articles/2491525/how-can-i-repeat-a-transfer
7
https://wise.com/help/articles/2977993/can-i-send-from-a-joint-bank-account
8
https://wise.com/help/articles/2977990/can-i-send-from-someone-elses-bank-account
9
https://wise.com/help/articles/2452305/how-can-i-check-the-status-of-my-transfer
10
https://wise.com/help/articles/2941900/when-will-my-money-arrive
11
https://wise.com/help/articles/2977950/why-does-it-say-my-transfers-complete-when-the-money-hasnt-arrived-yet
12
https://wise.com/help/articles/2977951/why-is-my-transfer-ta

In [1]:

statuses = {
  "EXCLUDE": {
    "label": "EXCLUDED BY PACTUM",
    "order": 2015,
    "analyticsTag": "NOT_APPLICABLE"
  },
  "ONGOING": {
    "label": "STAGING",
    "order": 500,
    "analyticsTag": "ONGOING"
  },
  "DECLINED": {
    "label": "DECLINED",
    "order": 2000,
    "analyticsTag": "DECLINED_CONCLUDED"
  },
  "LINK_EMAIL_1": {
    "label": "1ST LINK EMAIL SENT",
    "order": 301,
    "analyticsTag": "LINK_SENT"
  },
  "LINK_EMAIL_2": {
    "label": "2ND LINK EMAIL SENT",
    "order": 302,
    "analyticsTag": "LINK_SENT"
  },
  "LINK_EMAIL_3": {
    "label": "3RD LINK EMAIL SENT",
    "order": 303,
    "analyticsTag": "LINK_SENT"
  },
  "ONGOING_HELP": {
    "label": "SUPPLIER HAS WRITTEN FEEDBACK IN PORTAL - HELP BUTTON",
    "order": 520,
    "analyticsTag": "ONGOING"
  },
  "ONGOING_WAIT": {
    "label": " A WEEK PAUSE",
    "order": 600,
    "analyticsTag": "ONGOING"
  },
  "CLIENT_SIGNED": {
    "label": "CLIENT SIGNED IN DOCUSIGN",
    "order": 2030,
    "analyticsTag": "AGREEMENT_CONCLUDED"
  },
  "CONTACT_EMAIL_1": {
    "label": "1ST CONTACT CONFIRMATION EMAIL SENT",
    "order": 101,
    "analyticsTag": "REACHED_OUT"
  },
  "CONTACT_EMAIL_2": {
    "label": "2ND CONTACT CONFIRMATION EMAIL SENT",
    "order": 102,
    "analyticsTag": "REACHED_OUT"
  },
  "ONGOING_EMAIL_1": {
    "label": "1ST ONGOING EMAIL SENT",
    "order": 501,
    "analyticsTag": "ONGOING"
  },
  "ONGOING_EMAIL_2": {
    "label": "2ND ONGOING EMAIL SENT",
    "order": 502,
    "analyticsTag": "ONGOING"
  },
  "ONGOING_EMAIL_3": {
    "label": "3RD ONGOING EMAIL SENT",
    "order": 503,
    "analyticsTag": "ONGOING"
  },
  "SUPPLIER_SIGNED": {
    "label": "SUPPLIER SIGNED IN DOCUSIGN",
    "order": 2025,
    "analyticsTag": "AGREEMENT_CONCLUDED"
  },
  "LINK_EMAIL_FINAL": {
    "label": "FINAL LINK EMAIL SENT",
    "order": 304,
    "analyticsTag": "LINK_SENT"
  },
  "SENT_FOR_SIGNING": {
    "label": "DOCUSIGN SENT TO SUPPLIER",
    "order": 810,
    "analyticsTag": "AGREEMENT_REACHED"
  },
  "AGREEMENT_REACHED": {
    "label": "AGREEMENT REACHED IN PORTAL",
    "order": 800,
    "analyticsTag": "AGREEMENT_REACHED"
  },
  "BLOCKED_BY_CLIENT": {
    "label": "CLIENT SAID NOT TO USE THIS SUPPLIER",
    "order": 2020,
    "analyticsTag": "BLOCKED"
  },
  "CONTACT_CONFIRMED": {
    "label": "SUPPLIER CONFIRMED CONTACT",
    "order": 300,
    "analyticsTag": "REACHED_OUT"
  },
  "ONGOING_WEEK_WAIT": {
    "label": "ONGOING_WEEK_WAIT",
    "order": 602,
    "analyticsTag": "ONGOING"
  },
  "READY_FOR_SIGNING": {
    "label": "DOCUSIGN ENVELOPE CREATED",
    "order": 801,
    "analyticsTag": "AGREEMENT_REACHED"
  },
  "ONGOING_SCOPE_WAIT": {
    "label": "ONGOING_SCOPE_WAIT",
    "order": 572,
    "analyticsTag": "ONGOING"
  },
  "ONGOING_TERMS_WAIT": {
    "label": "ONGOING_TERMS_WAIT",
    "order": 542,
    "analyticsTag": "ONGOING"
  },
  "SENT_FOR_SIGNING_1": {
    "label": "1ST SENT FOR SIGNING EMAIL SENT",
    "order": 811,
    "analyticsTag": "AGREEMENT_REACHED"
  },
  "SENT_FOR_SIGNING_2": {
    "label": "2ND SENT FOR SIGNING EMAIL SENT",
    "order": 812,
    "analyticsTag": "AGREEMENT_REACHED"
  },
  "CONTACT_EMAIL_FINAL": {
    "label": "FINAL CONTACT CONFIRMATION EMAIL SENT",
    "order": 104,
    "analyticsTag": "REACHED_OUT"
  },
  "ONGOING_EMAIL_FINAL": {
    "label": "FINAL ONGOING EMAIL SENT",
    "order": 504,
    "analyticsTag": "ONGOING"
  },
  "ONGOING_TERMS_WRONG": {
    "label": "PAYMENT TERMS INCORRECT",
    "order": 540,
    "analyticsTag": "ONGOING"
  },
  "CONTACT_EMAIL_SEND_0": {
    "label": "STAGING",
    "order": 100,
    "analyticsTag": "REACHED_OUT"
  },
  "DECLINED_NO_RESPONSE": {
    "label": "DECLINED NO RESPONSE - MANUAL ONLY",
    "order": 2010,
    "analyticsTag": "DECLINED_CONCLUDED"
  },
  "ONGOING_PRICING_WAIT": {
    "label": "ONGOING_PRICING_WAIT",
    "order": 552,
    "analyticsTag": "ONGOING"
  },
  "ONGOING_CAPACITY_WAIT": {
    "label": "ONGOING_CAPACITY_WAIT",
    "order": 562,
    "analyticsTag": "ONGOING"
  },
  "ONGOING_PRICING_WRONG": {
    "label": "PRICING INCORRECT",
    "order": 550,
    "analyticsTag": "ONGOING"
  },
  "CONTACT_NOT_RESPONDING": {
    "label": "CONTACT HAS NOT RESPONDED",
    "order": 105,
    "analyticsTag": "REACHED_OUT"
  },
  "ONGOING_CAPACITY_WRONG": {
    "label": "ONGOING CAPACITY WRONG",
    "order": 560,
    "analyticsTag": "ONGOING"
  },
  "ONGOING_SCOPE_DECLINED": {
    "label": "SUPPLIER DECLINED NEW SCOPE OF WORK",
    "order": 570,
    "analyticsTag": "ONGOING"
  },
  "ONGOING_WAIT_CONFIRMED": {
    "label": "ONGOING_WAIT_CONFIRMED",
    "order": 604,
    "analyticsTag": "ONGOING"
  },
  "SENT_FOR_SIGNING_FINAL": {
    "label": "FINAL SENT FOR SIGNING EMAIL SENT",
    "order": 813,
    "analyticsTag": "AGREEMENT_REACHED"
  },
  "ONGOING_SCOPE_CONFIRMED": {
    "label": "ONGOING_SCOPE_CONFIRMED",
    "order": 574,
    "analyticsTag": "ONGOING"
  },
  "ONGOING_TERMS_CONFIRMED": {
    "label": "ONGOING_TERMS_CONFIRMED",
    "order": 544,
    "analyticsTag": "ONGOING"
  },
  "ALTERNATE_CONTACT_SEND_0": {
    "label": "STAGING",
    "order": 109,
    "analyticsTag": "REACHED_OUT"
  },
  "ALTERNATE_CONTACT_EMAIL_1": {
    "label": "1ST ALTERNATE CONTACT CONFIRMATION EMAIL SENT",
    "order": 110,
    "analyticsTag": "REACHED_OUT"
  },
  "ALTERNATE_CONTACT_EMAIL_2": {
    "label": "2ND ALTERNATE CONTACT CONFIRMATION EMAIL SENT",
    "order": 111,
    "analyticsTag": "REACHED_OUT"
  },
  "DECLINED_DEADLINE_EXPIRED": {
    "label": "DECLINED NO RESPONSE FROM LINK",
    "order": 2005,
    "analyticsTag": "DECLINED_CONCLUDED"
  },
  "ONGOING_PRICING_CONFIRMED": {
    "label": "ONGOING_PRICING_CONFIRMED",
    "order": 554,
    "analyticsTag": "ONGOING"
  },
  "ONGOING_CAPACITY_CONFIRMED": {
    "label": "ONGOING_CAPACITY_CONFIRMED",
    "order": 564,
    "analyticsTag": "ONGOING"
  },
  "CONTACT_NOT_RESPONDING_FINAL": {
    "label": "CONTACT HAS NOT RESPONDED FINAL",
    "order": 106,
    "analyticsTag": "REACHED_OUT"
  },
  "SENT_FOR_SIGNING_NO_RESPONSE": {
    "label": "SUPPLIER DID NOT SIGN AGREEMENT",
    "order": 815,
    "analyticsTag": "AGREEMENT_CONCLUDED"
  },
  "ALTERNATE_CONTACT_EMAIL_FINAL": {
    "label": "FINAL ALTERNATE CONTACT CONFIRMATION EMAIL SENT",
    "order": 112,
    "analyticsTag": "REACHED_OUT"
  },
  "CONTACT_CONFIRMATION_REJECTED": {
    "label": "CONTACT CONFIRMATION REJECTED",
    "order": 120,
    "analyticsTag": "REACHED_OUT"
  },
  "CONTACT_CONFIRMATION_NEW_CONTACT": {
    "label": "SUPPLIER SUBMITTED NEW CONTACT",
    "order": 130,
    "analyticsTag": "REACHED_OUT"
  }
}
statuses

{'EXCLUDE': {'label': 'EXCLUDED BY PACTUM',
  'order': 2015,
  'analyticsTag': 'NOT_APPLICABLE'},
 'ONGOING': {'label': 'STAGING', 'order': 500, 'analyticsTag': 'ONGOING'},
 'DECLINED': {'label': 'DECLINED',
  'order': 2000,
  'analyticsTag': 'DECLINED_CONCLUDED'},
 'LINK_EMAIL_1': {'label': '1ST LINK EMAIL SENT',
  'order': 301,
  'analyticsTag': 'LINK_SENT'},
 'LINK_EMAIL_2': {'label': '2ND LINK EMAIL SENT',
  'order': 302,
  'analyticsTag': 'LINK_SENT'},
 'LINK_EMAIL_3': {'label': '3RD LINK EMAIL SENT',
  'order': 303,
  'analyticsTag': 'LINK_SENT'},
 'ONGOING_HELP': {'label': 'SUPPLIER HAS WRITTEN FEEDBACK IN PORTAL - HELP BUTTON',
  'order': 520,
  'analyticsTag': 'ONGOING'},
 'ONGOING_WAIT': {'label': ' A WEEK PAUSE',
  'order': 600,
  'analyticsTag': 'ONGOING'},
 'CLIENT_SIGNED': {'label': 'CLIENT SIGNED IN DOCUSIGN',
  'order': 2030,
  'analyticsTag': 'AGREEMENT_CONCLUDED'},
 'CONTACT_EMAIL_1': {'label': '1ST CONTACT CONFIRMATION EMAIL SENT',
  'order': 101,
  'analyticsTag': '

In [3]:
# create a set of analyticsTags
analyticsTags = set()
for status in statuses:
    analyticsTags.add(statuses[status]['analyticsTag'])
analyticsTags

{'AGREEMENT_CONCLUDED',
 'AGREEMENT_REACHED',
 'BLOCKED',
 'DECLINED_CONCLUDED',
 'LINK_SENT',
 'NOT_APPLICABLE',
 'ONGOING',
 'REACHED_OUT'}